In [1]:
!pip install -q wurlitzer ninja
%load_ext wurlitzer

In [2]:
import os, time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline
from torchvision import datasets, transforms

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = "cuda"

torch.manual_seed(290)

# Conv + Relu Fused Py Kernel

In [3]:
import torch
import torch.nn.functional as F

def run_kernel(f, times, *args):
    for i in range(times):
        f(i, *args)

In [4]:
# 🔹 Description:
# This function is supposed to perform a 3x3 convolution on each pixel of the input image
# and then apply the ReLU activation function to the result.
#
# In other words, this is the "kernel" part — where the actual multiply–accumulate
# operations of convolution happen.
#
# Parameters:
# - i : the index of the pixel (from 0 to N*H*W - 1)
# - x : the input tensor of shape [N, 1, H, W]
# - w : the convolution filter (weights) of shape [1, 1, 3, 3]
# - b : the bias term (a scalar tensor)
# - out : the output tensor to store results
# - N, H, W : dimensions of the input (batch size, height, width)
#
# Inside this function, students should:
# 1️ Convert i into (n, h, w) indices — to locate the correct pixel in the batch
# 2️ Compute the accumulated sum (acc) by multiplying the 3x3 neighborhood by the weights
# 3️ Add the bias term b
# 4️ Apply ReLU (if acc < 0, set it to 0)
# 5️ Store the result in out[n, 0, h, w]

def conv_relu_kernel_py(i, x, w, b, out, N, H, W):
    if i >= N * H * W:
        return
    tmp_i = i
    i_n, tmp_i = divmod(tmp_i, H*W)
    i_h, i_w = divmod(tmp_i, W)

    local_pos = [-1,0,1]

    out[i_n][0][i_h][i_w] = b[0]
    for pos_y in range(3):
        inp_pos_y = local_pos[pos_y] + i_h
        if not 0 <= inp_pos_y < H:
            continue
        for pos_x in range(3):
            inp_pos_x = local_pos[pos_x] + i_w
            if not 0 <= inp_pos_x < W:
                continue
            out[i_n][0][i_h][i_w] += w[0][0][pos_y][pos_x] * x[i_n][0][inp_pos_y][inp_pos_x]

    out[i_n][0][i_h][i_w] = max(0, out[i_n][0][i_h][i_w])

In [5]:
# Function: conv_relu_py
# ----------------------
# 🔹 Description:
# This is the higher-level wrapper function that coordinates the operation.
# It:
# 1️ Checks that the input tensors have the expected shapes (using assert)
# 2️ Creates an output tensor with the same size as x
# 3️ Calls the run_kernel function, which runs conv_relu_kernel_py
#     for each pixel index i (from 0 to N*H*W)
# 4️ Returns the output tensor
#
# So, this function organizes and launches the lower-level computation.
def conv_relu_py(x, w, b):
    out = torch.empty_like(x)
    N = x.shape[0]
    assert 1 == x.shape[1]
    H = x.shape[2]
    W = x.shape[3]
    run_kernel(conv_relu_kernel_py, x.numel(), x, w, b, out, N, H, W)
    return out

In [6]:
device = "cpu"  

H, W = 8, 8  

x = torch.randn(4, 1, H, W, device=device)

conv = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=True).to(device)

with torch.no_grad():
    conv.weight.copy_(torch.randn_like(conv.weight))
    conv.bias.copy_(torch.randn_like(conv.bias))

w = conv.weight.detach()
b = conv.bias.detach()

y_ref = F.relu(conv(x))

y_py = conv_relu_py(x, w, b)

print("Max diff:", (y_ref - y_py).abs().max().item())


Max diff: 9.5367431640625e-07


# Cuda Kernel

In [7]:
cuda_begin = r'''
#include <torch/extension.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
'''

cuda_src = cuda_begin + r'''

__global__ void conv_relu_kernel(
    const float* __restrict__ x,
    const float* __restrict__ w,
    const float* __restrict__ b,
    float* __restrict__ out,
    int N,
    int H,
    int W
) {
    const int i = blockIdx.x * blockDim.x + threadIdx.x;
    const int num_pixels_total = N * H * W;
    if (i >= num_pixels_total) {
        return;
    }
    const int i_w = i % W;
    const int i_h = (i / W) % H;
    const int i_n = i / (H * W);
    float acc = b[0];
    for (int pos_y = 0; pos_y < 3; ++pos_y) {
        const int inp_pos_y = (pos_y - 1) + i_h; 
        for (int pos_x = 0; pos_x < 3; ++pos_x) {
            const int inp_pos_x = (pos_x - 1) + i_w;
            if (inp_pos_y >= 0 && inp_pos_y < H && inp_pos_x >= 0 && inp_pos_x < W) {
                const int w_idx = pos_y * 3 + pos_x;
                const int x_idx = (i_n * H * W) + (inp_pos_y * W) + inp_pos_x;
                acc += w[w_idx] * x[x_idx];
            }
        }
    }
    out[i] = fmaxf(0.0f, acc);
}

torch::Tensor conv_relu_fused(torch::Tensor x,
                              torch::Tensor w,
                              torch::Tensor b) {
    CHECK_INPUT(x);
    CHECK_INPUT(w);
    CHECK_INPUT(b);

    TORCH_CHECK(x.dim() == 4, "x must be [N,C,H,W]");
    TORCH_CHECK(w.dim() == 4, "w must be [C_out,C_in,3,3]");
    TORCH_CHECK(b.dim() == 1, "b must be [C_out]");

    TORCH_CHECK(x.size(1) == 1, "only C_in=1 supported");
    TORCH_CHECK(w.size(0) == 1 && w.size(1) == 1 &&
                w.size(2) == 3 && w.size(3) == 3,
                "only 1x1x3x3 kernel supported");
    TORCH_CHECK(b.size(0) == 1, "only 1 output channel supported");

    auto x_c = x.contiguous();
    auto w_c = w.contiguous();
    auto b_c = b.contiguous();

    int N = x_c.size(0);
    int H = x_c.size(2);
    int W = x_c.size(3);

    auto out = torch::empty_like(x_c);

    int n_pix = N * H * W;
    int threads = 256;
    int blocks = cdiv(n_pix, threads);

    conv_relu_kernel<<<blocks, threads>>>(
        x_c.data_ptr<float>(),
        w_c.data_ptr<float>(),
        b_c.data_ptr<float>(),
        out.data_ptr<float>(),
        N, H, W
    );
    C10_CUDA_KERNEL_LAUNCH_CHECK();

    return out;
}
'''

cpp_src = r'''
torch::Tensor conv_relu_fused(torch::Tensor x,
                              torch::Tensor w,
                              torch::Tensor b);
'''

module = load_inline(
    name="conv_relu_fused_ext",
    cpp_sources=[cpp_src],
    cuda_sources=[cuda_src],
    functions=["conv_relu_fused"],
    extra_cuda_cflags=["-O3"],
    verbose=False,
)


/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [8]:
class FusedConvReLUFn(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, x, weight, bias):

        assert x.is_cuda and weight.is_cuda and bias.is_cuda
        assert x.dim() == 4 and x.size(1) == 1, "only C_in=1 supported"
        assert weight.shape == (1, 1, 3, 3), "only 1x1x3x3 kernel supported"
        assert bias.shape == (1,), "only 1 output channel supported"

        y = module.conv_relu_fused(x, weight, bias)
        ctx.save_for_backward(x, weight, bias, y)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        x, weight, bias, y = ctx.saved_tensors

        # grad ReLU
        mask = (y > 0).to(grad_output.dtype)
        grad_z = grad_output * mask

        # dL/dx
        grad_x = torch.nn.grad.conv2d_input(
            x.shape, weight, grad_z, padding=1
        )
        # dL/dW
        grad_weight = torch.nn.grad.conv2d_weight(
            x, weight.shape, grad_z, padding=1
        )
        # dL/db
        grad_bias = grad_z.sum(dim=[0, 2, 3])

        return grad_x, grad_weight, grad_bias


In [9]:
class FusedConvReLU(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1, 1, 3, 3))
        self.bias   = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return FusedConvReLUFn.apply(x, self.weight, self.bias)


In [10]:
class CNNBaseline(nn.Module):
    def __init__(self, num_convs=5):
        super().__init__()

        self.num_convs = num_convs
        self.conv = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(14*14, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        
        for _ in range(self.num_convs):
            x = F.relu(self.conv(x))
        
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [11]:
class CNNFused(nn.Module):
    def __init__(self, num_convs=5):
        super().__init__()
        
        self.num_convs = num_convs
        self.conv = FusedConvReLU()
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(14*14, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):

        for _ in range(self.num_convs):
            x = self.conv(x)
        
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [12]:
batch_size = 128

transform = transforms.ToTensor()

train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = torch.utils.data.DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 38.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.04MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.78MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.69MB/s]


In [13]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [14]:
def train_model_timed(model, train_loader, test_loader, device, epochs=3, lr=1e-3):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    epoch_total_times   = []
    epoch_forward_times = []
    epoch_backward_times= []
    epoch_other_times   = []
    test_accuracies     = []

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0

        fwd_time = 0.0
        bwd_time = 0.0
        other_time = 0.0

        torch.cuda.synchronize()
        epoch_start = time.time()

        for images, labels in train_loader:
            step_start = time.time()

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            # ----- forward -----
            torch.cuda.synchronize()
            t0 = time.time()
            outputs = model(images)
            loss = criterion(outputs, labels)
            torch.cuda.synchronize()
            t1 = time.time()
            fwd_time += (t1 - t0)

            # ----- backward -----
            torch.cuda.synchronize()
            t2 = time.time()
            optimizer.zero_grad()
            loss.backward()
            torch.cuda.synchronize()
            t3 = time.time()
            bwd_time += (t3 - t2)

            # ----- optimizer step -----
            torch.cuda.synchronize()
            t4 = time.time()
            optimizer.step()
            torch.cuda.synchronize()
            t5 = time.time()
            other_time += (t5 - t4)

            running_loss += loss.item() * labels.size(0)

        torch.cuda.synchronize()
        epoch_end = time.time()
        epoch_time = epoch_end - epoch_start

        avg_loss = running_loss / len(train_loader.dataset)
        acc = evaluate(model, test_loader, device)

        epoch_total_times.append(epoch_time)
        epoch_forward_times.append(fwd_time)
        epoch_backward_times.append(bwd_time)
        epoch_other_times.append(other_time)
        test_accuracies.append(acc)

        print(
            f"Epoch {epoch}: "
            f"loss={avg_loss:.4f}, "
            f"test_acc={acc*100:.2f}%, "
            f"time_total={epoch_time:.2f}s "
            f"(fwd={fwd_time:.2f}s, bwd={bwd_time:.2f}s, other={other_time:.2f}s)"
        )

    return {
        "total_times": epoch_total_times,
        "forward_times": epoch_forward_times,
        "backward_times": epoch_backward_times,
        "other_times": epoch_other_times,
        "test_accuracies": test_accuracies,
    }


## num_convs = 10

In [15]:
num_convs = 10 # 2 and 10 should be tested

baseline_cudnn = CNNBaseline(num_convs).to(device)
baseline_cudnn_off = CNNBaseline(num_convs).to(device)
baseline_cpu = CNNBaseline(num_convs).to(device)
fused = CNNFused(num_convs).to(device)

In [16]:
def copy_weights(dst, src):
    with torch.no_grad():
        
        dst.conv.weight.copy_(src.conv.weight)
        dst.conv.bias.copy_(src.conv.bias)
        dst.fc1.weight.copy_(src.fc1.weight)
        dst.fc1.bias.copy_(src.fc1.bias)
        dst.fc2.weight.copy_(src.fc2.weight)
        dst.fc2.bias.copy_(src.fc2.bias)

copy_weights(fused, baseline_cudnn)
copy_weights(baseline_cudnn_off, baseline_cudnn)
copy_weights(baseline_cpu, baseline_cudnn)

In [17]:
print("=== Training baseline CNN (Conv2d + ReLU) - cuDNN  ===")
device = "cuda"

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

stats_base1 = train_model_timed(baseline_cudnn, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - cuDNN  ===
Epoch 1: loss=2.3020, test_acc=11.35%, time_total=7.21s (fwd=2.39s, bwd=3.23s, other=0.53s)
Epoch 2: loss=2.3015, test_acc=11.35%, time_total=5.29s (fwd=1.17s, bwd=2.67s, other=0.36s)
Epoch 3: loss=2.3014, test_acc=11.35%, time_total=5.36s (fwd=1.20s, bwd=2.71s, other=0.38s)
Epoch 4: loss=2.3014, test_acc=11.35%, time_total=5.29s (fwd=1.19s, bwd=2.70s, other=0.37s)
Epoch 5: loss=2.3014, test_acc=11.35%, time_total=5.42s (fwd=1.09s, bwd=2.57s, other=0.35s)


In [18]:
print("\n=== Training fused CNN (FusedConvReLU) ===")

stats_fused = train_model_timed(fused, train_loader, test_loader, device, epochs=5)



=== Training fused CNN (FusedConvReLU) ===
Epoch 1: loss=2.3019, test_acc=11.35%, time_total=5.82s (fwd=0.64s, bwd=3.69s, other=0.31s)
Epoch 2: loss=2.3014, test_acc=11.35%, time_total=5.72s (fwd=0.70s, bwd=3.87s, other=0.33s)
Epoch 3: loss=2.3014, test_acc=11.35%, time_total=5.76s (fwd=0.67s, bwd=3.79s, other=0.32s)
Epoch 4: loss=2.3014, test_acc=11.35%, time_total=5.81s (fwd=0.71s, bwd=3.95s, other=0.33s)
Epoch 5: loss=2.3014, test_acc=11.35%, time_total=5.76s (fwd=0.69s, bwd=3.87s, other=0.32s)


In [19]:
print("=== Training baseline CNN (Conv2d + ReLU) - cuDNN off ===")

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False

stats_base2 = train_model_timed(baseline_cudnn_off, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - cuDNN off ===
Epoch 1: loss=2.3020, test_acc=11.35%, time_total=63.86s (fwd=19.23s, bwd=44.10s, other=0.27s)
Epoch 2: loss=2.3014, test_acc=11.35%, time_total=63.77s (fwd=19.26s, bwd=43.98s, other=0.27s)
Epoch 3: loss=2.3014, test_acc=11.35%, time_total=64.39s (fwd=19.82s, bwd=44.03s, other=0.27s)
Epoch 4: loss=2.3014, test_acc=11.35%, time_total=64.23s (fwd=19.56s, bwd=44.14s, other=0.27s)
Epoch 5: loss=2.3014, test_acc=11.35%, time_total=63.84s (fwd=19.21s, bwd=44.08s, other=0.27s)


In [20]:
print("=== Training baseline CNN (Conv2d + ReLU) - CPU ===")
device = "cpu"

stats_base3 = train_model_timed(baseline_cpu, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - CPU ===
Epoch 1: loss=2.3020, test_acc=11.35%, time_total=27.09s (fwd=4.13s, bwd=22.31s, other=0.41s)
Epoch 2: loss=2.3015, test_acc=11.35%, time_total=26.42s (fwd=4.11s, bwd=21.69s, other=0.38s)
Epoch 3: loss=2.3014, test_acc=11.35%, time_total=26.64s (fwd=3.98s, bwd=22.04s, other=0.38s)
Epoch 4: loss=2.3013, test_acc=11.35%, time_total=26.59s (fwd=4.21s, bwd=21.77s, other=0.38s)
Epoch 5: loss=2.3014, test_acc=11.35%, time_total=26.46s (fwd=4.13s, bwd=21.72s, other=0.38s)


## num_convs = 2

In [21]:
num_convs = 2 # 2 and 10 should be tested

baseline_cudnn = CNNBaseline(num_convs).to(device)
baseline_cudnn_off = CNNBaseline(num_convs).to(device)
baseline_cpu = CNNBaseline(num_convs).to(device)
fused = CNNFused(num_convs).to(device)

In [22]:
copy_weights(fused, baseline_cudnn)
copy_weights(baseline_cudnn_off, baseline_cudnn)
copy_weights(baseline_cpu, baseline_cudnn)

In [23]:
print("=== Training baseline CNN (Conv2d + ReLU) - cuDNN  ===")
device = "cuda"

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

stats_base1 = train_model_timed(baseline_cudnn, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - cuDNN  ===
Epoch 1: loss=0.9193, test_acc=89.92%, time_total=4.96s (fwd=0.43s, bwd=0.93s, other=0.33s)
Epoch 2: loss=0.3205, test_acc=92.32%, time_total=4.78s (fwd=0.46s, bwd=0.98s, other=0.37s)
Epoch 3: loss=0.2642, test_acc=93.08%, time_total=4.95s (fwd=0.46s, bwd=1.00s, other=0.35s)
Epoch 4: loss=0.2333, test_acc=93.81%, time_total=5.12s (fwd=0.44s, bwd=0.95s, other=0.33s)
Epoch 5: loss=0.2122, test_acc=94.14%, time_total=4.87s (fwd=0.48s, bwd=1.01s, other=0.36s)


In [24]:
print("\n=== Training fused CNN (FusedConvReLU) ===")

stats_fused = train_model_timed(fused, train_loader, test_loader, device, epochs=5)



=== Training fused CNN (FusedConvReLU) ===
Epoch 1: loss=0.9105, test_acc=89.33%, time_total=4.88s (fwd=0.38s, bwd=1.29s, other=0.34s)
Epoch 2: loss=0.3326, test_acc=91.74%, time_total=5.11s (fwd=0.38s, bwd=1.29s, other=0.34s)
Epoch 3: loss=0.2759, test_acc=92.76%, time_total=4.97s (fwd=0.37s, bwd=1.26s, other=0.34s)
Epoch 4: loss=0.2442, test_acc=93.49%, time_total=4.98s (fwd=0.37s, bwd=1.26s, other=0.34s)
Epoch 5: loss=0.2215, test_acc=94.12%, time_total=5.03s (fwd=0.38s, bwd=1.29s, other=0.35s)


In [25]:
print("=== Training baseline CNN (Conv2d + ReLU) - cuDNN off ===")

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False

stats_base2 = train_model_timed(baseline_cudnn_off, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - cuDNN off ===
Epoch 1: loss=0.9218, test_acc=89.88%, time_total=13.94s (fwd=4.45s, bwd=8.98s, other=0.25s)
Epoch 2: loss=0.3362, test_acc=91.90%, time_total=13.68s (fwd=4.63s, bwd=8.56s, other=0.25s)
Epoch 3: loss=0.2820, test_acc=92.45%, time_total=13.85s (fwd=4.67s, bwd=8.67s, other=0.25s)
Epoch 4: loss=0.2495, test_acc=93.22%, time_total=14.06s (fwd=4.83s, bwd=8.73s, other=0.25s)
Epoch 5: loss=0.2253, test_acc=93.79%, time_total=14.40s (fwd=4.94s, bwd=8.94s, other=0.24s)


In [26]:
print("=== Training baseline CNN (Conv2d + ReLU) - CPU ===")
device = "cpu"

stats_base3 = train_model_timed(baseline_cpu, train_loader, test_loader, device, epochs=5)

=== Training baseline CNN (Conv2d + ReLU) - CPU ===
Epoch 1: loss=0.9217, test_acc=89.57%, time_total=7.21s (fwd=1.31s, bwd=4.17s, other=0.54s)
Epoch 2: loss=0.3389, test_acc=91.13%, time_total=6.75s (fwd=1.36s, bwd=4.50s, other=0.43s)
Epoch 3: loss=0.2862, test_acc=92.25%, time_total=6.59s (fwd=1.35s, bwd=4.59s, other=0.40s)
Epoch 4: loss=0.2517, test_acc=93.11%, time_total=6.92s (fwd=1.30s, bwd=4.28s, other=0.51s)
Epoch 5: loss=0.2267, test_acc=93.76%, time_total=6.75s (fwd=1.35s, bwd=4.49s, other=0.46s)
